# Testing out how to extract landmarks and passing it out to the model

In [4]:
import sys
sys.path.append('..')

In [5]:
import numpy as np

In [6]:
from app.api.utils.mediapipe_utils import process_video

In [7]:
landmarks = process_video('../squat.mp4')

I0000 00:00:1730393467.455292 2496907 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1730393467.560944 2652902 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 560.35.03), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
W0000 00:00:1730393467.616608 2652901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730393467.633860 2652892 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/dele/.pyenv/versions/3.11.4/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [8]:
print(landmarks)

[[0, 621, 745, 624, 757, 621, 745, 620, 669, 620, 669, 652, 614, 624, 757, 550, 694, 550, 694, 640, 620, 596, 669, 621, 745, 631, 689, 624, 757, 621, 745, 624, 757], [3, 693, 812, 690, 805, 693, 812, 568, 843, 568, 843, 430, 837, 690, 805, 682, 778, 682, 778, 607, 687, 680, 697, 693, 812, 642, 721, 690, 805, 693, 812, 690, 805], [5, 613, 1375, 719, 1371, 613, 1375, 449, 1612, 449, 1612, 444, 1917, 719, 1371, 740, 1641, 740, 1641, 832, 1904, 530, 942, 613, 1375, 760, 920, 719, 1371, 613, 1375, 719, 1371], [6, 606, 1420, 741, 1415, 606, 1420, 447, 1611, 447, 1611, 441, 1914, 741, 1415, 799, 1626, 799, 1626, 832, 1901, 530, 972, 606, 1420, 754, 953, 741, 1415, 606, 1420, 741, 1415], [7, 605, 1455, 747, 1447, 605, 1455, 446, 1603, 446, 1603, 439, 1912, 747, 1447, 805, 1626, 805, 1626, 835, 1901, 530, 1008, 605, 1455, 753, 995, 747, 1447, 605, 1455, 747, 1447], [8, 601, 1492, 743, 1478, 601, 1492, 446, 1599, 446, 1599, 434, 1903, 743, 1478, 814, 1614, 814, 1614, 835, 1896, 533, 1062, 601, 1

## Importing model and testing out the prediction

In [10]:
from tensorflow.keras.models import load_model

In [11]:
model = load_model('../app/models/squat_model.keras')

I0000 00:00:1730393496.244819 2496907 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-31 11:51:36.261726: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
input_data = np.expand_dims(landmarks, axis=0)  # Shape should be (1, frames, features)

In [13]:
prediction = model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step


In [14]:
print(prediction)

[[0.975406]]


In [59]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [62]:
import pandas as pd

In [77]:
def process_csv(data, sequence_length=60):
    data = np.array(data)

    # Pad or truncate the sequence to match the expected input size (SEQUENCE_LENGTH)
    if len(data) > sequence_length:
        data = data[:sequence_length]
    elif len(data) < sequence_length:
        padding = np.zeros((sequence_length - len(data), data.shape[1]))
        data = np.vstack((data, padding))

    # Reshape data to match model input shape: (1, SEQUENCE_LENGTH, number_of_features)
    data = np.expand_dims(data, axis=0)

    return data

In [78]:
vid_data = process_csv(landmarks)

In [79]:
model.predict(vid_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([[0.975406]], dtype=float32)